In [23]:
import yfinance as yf
import datetime
import requests
import urllib.request
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


In [25]:
def get_data(symbols, start, end):
    start = start.split("/")
    end = end.split("/")
    start = datetime.datetime(int(start[0]), int(start[1]), int(start[2]))
    end = datetime.datetime(int(end[0]), int(end[1]), int(end[2]))
    return yf.download(symbols, start=start, end=end)

def get_symbols(url, tag="a", attr="", fromText=False):
    response = requests.get(url)
    if not response:
        print("Status code > 400, web connection failed.")
        return None
    soup = BeautifulSoup(response.text, "html.parser")
    symbols = []
    for cur in soup.findAll(tag):
        if fromText:
            if cur.get_text() != None:
                symbols.append(cur.get_text())
        else:
            if cur.get(attr) != None:
                symbols.append(cur.get(attr))
    return symbols

def handle_one_df(data, symbol, cols):
    cur_col = [col for col in data.columns if symbol in col]
    return pd.DataFrame(np.array(data[cur_col]), columns=cols, index=data[cur_col].index)

def get_data_dict(symbols, data, cols, batch=10):
    df_dict = {}
    batch_steps = np.arange(0, len(symbols), batch)
    for i in range(1, len(batch_steps)+1):
        cur_symbols = symbols[batch_steps[i-1]:batch_steps[i]] if i != len(batch_steps) else symbols[batch_steps[i-1]:]
        for symbol in cur_symbols:
            df_dict[symbol] = handle_one_df(data, symbol, cols)
    print("Finished")
    return df_dict